In [ ]:
import pandas as pd
import numpy as np
import zipfile

#un peu de mise en forme
class color:
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'

----
###  Playing with all datasets

#### Loading dataset into dataframe objects

<font color="#0000ff">

Etapes :

1) On commence par nommer chaque objet Dataframe déjà loadé selon les différents jeux de données. On définit une liste contenant tous les dataframe. On peut commencer à regarder les types des colonnes grâce à dtypes.<br>
2) On construit la fonction convert_price en procédant au nettoyage de tous les caractères relatifs aux monnaies en output et en convertissant
ce dernier en float.<br>
3) La fonction preprocess complète le nettoyage et remplace sur les colonnes numériques les valeurs manquantes par la médiane.<br>
4) On preprocess chaque dataframe un par un en prenant soin de modifier le type des colonnes identifiées suite à dtypes sur les dataframe
amazon, topshop et victoria.<br>
</font>

In [ ]:
!ls data

In [ ]:
ae = pd.read_csv('data/ae_com.csv')
am = pd.read_csv('data/amazon_com.csv')
bt = pd.read_csv('data/btemptd_com.csv')
ck = pd.read_csv('data/calvinklein_com.csv')
hp = pd.read_csv('data/hankypanky_com.csv')
ma = pd.read_csv('data/macys_com.csv')
sn = pd.read_csv('data/shop_nordstrom_com.csv')
ts =pd.read_csv('data/us_topshop_com.csv')
vs = pd.read_csv('data/victoriassecret_com.csv')

list_data = [ae, am, bt, ck, hp, ma, sn, ts, vs]


In [ ]:
ae.dtypes

In [ ]:
vs.dtypes

In [ ]:
def convert_price(s):
    if s[:2]=='Rp':
        return float(s[2:])*0.000071
    elif s[0]=='₡':
        return float(s[1:])*0.0016
    elif s[-3:]=="USD":
        return float(s[:-3])
    elif '-'in s:
        from statistics import median
        return median([float(s[1:6]),float(s[-5:])])
    elif s[1]=='$':
        return float(s[2:6])
    else :
        return float(s[1:6])

In [ ]:
def preprocess_df(df, clean_fn, cols, cols2):
    df[cols] = df[cols].applymap(clean_fn)
    medians = df[cols].median()
    medians1= df[cols2].median()
    df.fillna(value=medians, inplace=True)
    df.fillna(value=medians1, inplace=True)

In [ ]:
preprocess_df(ae, convert_price, ['mrp', 'price'], ['review_count', 'rating'])

In [ ]:
preprocess_df(am, convert_price, ['mrp', 'price'], ['review_count', 'rating'])
am['review_count']=am['review_count'].astype('float64')

In [ ]:
preprocess_df(ck, convert_price, ['mrp', 'price'], ['review_count', 'rating'])

In [ ]:
preprocess_df(bt, convert_price, ['mrp', 'price'], ['review_count', 'rating'])

In [ ]:
preprocess_df(hp, convert_price, ['mrp', 'price'], ['review_count', 'rating'])

In [ ]:
preprocess_df(ma, convert_price, ['mrp', 'price'], ['review_count', 'rating'])

In [ ]:
preprocess_df(sn, convert_price, ['mrp', 'price'], ['review_count', 'rating'])

In [ ]:
preprocess_df(ts, convert_price, ['mrp', 'price'], ['review_count', 'rating'])
ts['style_attributes']=ts['style_attributes'].astype('object')

In [ ]:
preprocess_df(vs, convert_price, ['mrp', 'price'], ['review_count', 'rating'])
vs['style_attributes']=vs['style_attributes'].astype('object')

In [ ]:
df_all= pd.concat(list_data)
df_all

In [ ]:
len(ae) + len(am) + len(bt) + len(ck) + len(hp) + len(ma) + len(sn) + len(ts) + len(vs) == len(df_all)

Before having a look at the other dataset. 
To check for question 0 and 0bis, reload the df dataFrame from scratch 

**Question 1:** 

Check if for the Rp prices, we have a specific type of products sold. 

<font color="#0000ff">

Etapes :

1) Pour cette question, il faut reload les dataframe "from scratch" et ne pas les prétraiter.<br>
2) On construit une fonction simple qui identifie les catégories dont les prix sont en roupies.
</font>

In [ ]:
def rp(x):
    return 'Rp' in x

def roupies(listt):
    for dic in listt:
        if dic['price'].apply(rp).sum()>0:
            print(dic['retailer'][0]+" sells",color.BOLD,(dic['price'].apply(rp)==True).sum(),color.END," products in RP.\n")
            for s in set(dic.loc[dic['price'].apply(rp)==True]['product_category']):
                print(s,"\n")

In [ ]:
roupies(list_data)

Question 2:

Check for the factor to apply for currency converter to replace the prices by the USD equivalent.

Consider that:

    1 IRD (Roupie) = 0.000071 USD
    1 Costa Rica dollar = 0.0016 USD

Create a function that

    takes one argument to be one element from the price columns,
    identifies which currency it is,
    extracts the value and converts it to float.

Then apply your function to both price columns in df.

<font color="#0000ff">
Pour constuire cette fonction, nous avons d'abord parcouru les éléments de chaque dataframe et nous avons converti en float tous les prix qui ne comportaient pas l'équivalent USD; il était aussi
nécessaire, lorsque des tranches de prix apparaissaient, de faire la médiane de la tranche de prix afin de ne conserver qu'un seul nombre.
</font>

In [ ]:
def convert_price(s):
    if s[:2]=='Rp':
        return float(s[2:])*0.000071
    elif s[0]=='₡':
        return float(s[1:])*0.0016
    elif s[-3:]=="USD":
        return float(s[:-3])
    elif '-'in s:
        from statistics import median
        return median([float(s[1:6]),float(s[-5:])])
    elif s[1]=='$':
        return float(s[2:6])
    else :
        return float(s[1:6])

In [ ]:
for dic in list_data:
    dic[['price']].applymap(convert_price)

##### Question 3: 
Which online retailers sell some Victoria's Secret products? 

<font color="#0000ff">

Etapes :

1) Nous avons d'abord regardé quelles sont les différentes marques vendues par les retailers. Nous avons constaté que seuls Amazon et Victoria's Secret vendent des produits de
cette marque, et chacun comporte une écriture différente : "Victoria's Secret" et "Victorias-Secret". <br>
2) On construit une fonction simple qui prend en input la liste de dataframe puis parcourt chaque dataframe, compte toutes les fois où la marque 
Victoria's Secret est contenue dans la colonne brand_name, puis sort le retailer correspondant en output qui vend cette marque.<br>
</font>

In [ ]:
def retailers(listt):
    for dic in listt:
        brands= set(dic['brand_name'])
        print(color.BOLD, dic['retailer'][0], "sells the following brand(s) : \n", color.END )
        for s in brands :
            print(s, "\n")

In [ ]:
retailers(list_data)

In [ ]:
def fbis(x):
    for dic in x :
            if (dic.brand_name=="Victoria's Secret").sum()>0 or (dic.brand_name=="Victorias-Secret").sum()>0:
                print(color.BOLD,dic['retailer'][0]+" sells",(dic.brand_name=="Victoria's Secret").sum()+(dic.brand_name=="Victorias-Secret").sum()," Victoria's Secret products.\n", color.END)
            else:
                print(dic['retailer'][0]+ " doesn't sell Victoria's Secret products.\n")

In [ ]:
fbis(list_data)

---
**Question 4:**
What are the Victoria's Secret products available in the different retailers?


<font color="#0000ff">

Etapes :

1) Pour cette question, nous avons créé une fonction simple h qui prend un input un élément et cherche les str "Victoria's Secret" ou "Victorias-Secret".
En l'occurence, nous allons appliquer cette fonction à la colonne "brand name" grâce à la méthode apply. <br>
2) Nous définissons ensuite une fonction vicproducts qui pour chaque dataframe, retourne en output les produits de la marque VS vendus par ces retailers uniquement si la colonne brand name d'où on extrait les occurences Victoria est non vide.<br>
</font>

In [ ]:
def h(x):
    return "Victoria's Secret" in x or "Victorias-Secret" in x

def vicproducts(listt):
    for dic in listt:
        if len(set(dic.loc[dic['brand_name'].apply(h) == True]['product_name']))>0:
            print(color.BOLD, dic['retailer'][0], "sells the following product(s) : \n", color.END )
            for s in set(dic.loc[dic['brand_name'].apply(h) == True]['product_name']):
                print(s, "\n")

In [ ]:
vicproducts(list_data)

--- 
**Question 5:**
How many distinct Victoria's Secret products do we have here? (check 'product_name')

<font color="#0000ff">

Etapes :

1) Pour cette question, nous reprenons les processus précédents et le principe de la méthode apply. La seule différence est qu'on 
retourne en output le nombre distincts de produits; on utilise pour cela un set, avant de compter le nombre d'éléments du set grâce à la méthode len.
</font>

In [ ]:
def h(x):
    return "Victoria's Secret" in x or "Victorias-Secret" in x

def viccounts(listt):
    for dic in listt:
        if len(set(dic.loc[dic['brand_name'].apply(h) == True]['product_name']))>0:
            print(dic['retailer'][0], "sells",color.BOLD, len(set(dic.loc[dic['brand_name'].apply(h) == True]['product_name'])),color.END, "distinct product(s)")


In [ ]:
viccounts(list_data)

In [ ]:
a = 618
for s in set(vs.loc[vs['brand_name'].apply(h)== True]['product_name']):
    if "Victoria's Secret The Ultimate Maximum Support Sport Bra" in s:
        a = a-1
        break
print ('Number of Vs product is :', a)

<font color="#0000ff">
Ce que nous avons fait précédemment consiste à vérifier si le produit VS vendu par amazon, à savoir : "Victoria's Secret The Ultimate Maximum Support Sport Bra"
    est vendu par Victoria's Secret. On définit une variable a qui vaut 618, cad la somme des produits distincts amazon et victoria calculés plus tôt.
On définit une boucle for simple qui cherche le nom du produit dans la colonne équivalente des produits distincts de victoria, s'il le trouve on retire 1 à la variable a puis on arrête la boucle.<br>
Le résultat montre que le produit n'a pas été trouvé, par conséquent il ne se trouve pas dans le jeu de données Victoria's Secret
</font>

**Question 6:**
Try to merge amazon and VictoriaSecret datasets considering only Victorias-Secret products, using merge method from Pandas. 
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.merge.html?highlight=merge#pandas.DataFrame.merge

Compare prices.

<font color="#0000ff">

Pour cette question, il faut exécuter les questions 0 et 0 bis afin de prétraiter les dataframe vs et amazon: on cherche à ce que ces derniers soient homogéneisés (ce n'est pas forcément le cas d'un jeu de données à un autre). On cherche à merge nos dataframe en ne considérant uniquement que les produits Victoria's Secret, c'est pour cela qu'on crée un nouveau mini dataframe df_vsamazon extrait du dataframe amazon qui contient les données relatives au produit Victoria. 
</font>

In [ ]:
df_vsamazon= pd.DataFrame(am.loc[am['brand_name'].apply(h)==True])

In [ ]:
df_vsonly = pd.merge(vs, df_vsamazon, how='outer')
df_vsonly

<font color="#0000ff">
( ͡ಠ ʖ̯ ͡ಠ) Pour la question 'Compare price', on se rend compte qu'il n'est pas possible de répondre avec une analyse de prix 
puisque le produit Victoria's Secret vendu par Amazon ne l'est pas chez le retailer Victoria's Secret : on l'a observé 
dans la question 5 avec les produits distincts.
</font>

---

**Question 7:**
Explore which brands ('brand_name') are sold by each retailer 

<font color="#0000ff">
Même principe que pour la question 3, nous l'avions déjà faite en amont pour voir comment pouvait s'écrire la marque Victoria chez les différents retailers.
</font>

In [ ]:
def retailers(listt):
    for dic in listt:
        brands= set(dic['brand_name'])
        print(color.BOLD, dic['retailer'][0], "sells the following brand(s) : \n", color.END )
        for s in brands :
            print(s, "\n")

In [ ]:
retailers(list_data)

---

**Question 8:**
As we do not have so much Victoria's Secret products in the different files.

Explore the different files and the content for Calvin Klein products. 

Which retailers provide Calvin Klein products?


<font color="#0000ff">
Même principe que pour la question 3 bis.
</font>

In [ ]:
def fck(x):
    for dic in x :
            if (dic.brand_name=="Calvin Klein").sum()>0 or (dic.brand_name=="Calvin-Klein").sum()>0 or (dic.brand_name=="CALVIN KLEIN").sum()>0 or (dic.brand_name=="Calvin+Klein").sum()>0:
                print(color.BOLD,dic['retailer'][0]+" sells",(dic.brand_name=="Calvin Klein").sum()+(dic.brand_name=="Calvin-Klein").sum()+(dic.brand_name=="CALVIN KLEIN").sum()+(dic.brand_name=="Calvin+Klein").sum()," Calvin Klein products.\n", color.END)
            else:
                print(dic['retailer'][0]+ " doesn't sell Calvin Klein products.\n")
                

In [ ]:
fck(list_data)

----
**Question 9: Extraction and cleaning of Calvin Klein products data** 

Extract all data from each file for the Calvin Klein products. Check for the brand_name. It could be written in multiple ways. 

Put each in new dataframes. 

To do so: 
- Consider dataframes containing Clavin Klein products and preprocess each dataframe separately (it might be useful to adjust your preprocessing function when content is different)
- Identify and extract Calvin Klein product data

<font color="#0000ff">
Nos dataframes ont déjà été prétraités en amont du début des questions ; ainsi si nous n'avons pas intégré ce pan de la question c'est parce nous considérons qu'il faut déjà les prétraiter dans les questions 0 et 0 bis.
</font>

In [ ]:
def a(x):
    return "Calvin Klein" in x or "Calvin+Klein" in x or "Calvin-Klein" in x or "CALVIN KLEIN" in x

def ckcounts(listt):
    for dic in listt:
        if len(set(dic.loc[dic['brand_name'].apply(a) == True]['product_name']))>0:
            print(dic['retailer'][0], "sells",color.BOLD, len(set(dic.loc[dic['brand_name'].apply(a) == True]['product_name'])),color.END, "distinct product(s)")


In [ ]:
ckcounts(list_data)

In [ ]:
ck_ck= pd.DataFrame(ck.loc[ck['brand_name'].apply(a)==True])
am_ck= pd.DataFrame(am.loc[am['brand_name'].apply(a)==True])
ma_ck= pd.DataFrame(ma.loc[ma['brand_name'].apply(a)==True])
sn_ck= pd.DataFrame(sn.loc[sn['brand_name'].apply(a)==True])

In [ ]:
ck_products = [ck_ck, am_ck, ma_ck, sn_ck]

In [ ]:
ck_productss = pd.concat(ck_products)
ck_productss

Use the concat method to compile all extracted data into a new df_ck data frame. Also clean the price columns. Have a close look to the price columns.

You can also apply the fillna method applying medians (to be calculated again).

In [ ]:
len(ck_ck) + len(am_ck) + len(ma_ck) + len(sn_ck) == len(ck_productss)

**Question 10:**

Consider your Calvin Klein dataFrame. 

Extract the description of every product, only keep the unique description, and count which words appear the most. 

<font color="#0000ff">

Etapes :

1) Pour cette question, nous réalisons d'abord un set des descriptions correspondant aux produits Calvin Klein. En effet, il est important de ne garder que les descriptions disctinctes afin d'éviter les mots doublons.<br>
2) Ensuite, on utilise join qui assemble tous les éléments des descriptions, cad des mots, séparés par " ". On complète la manoeuvre en utilisant split pour retirer les espaces superflus. <br>
3) Enfin pour que ce soit plus simple, on compte tous les mots (il y en a 8814) et on affiche les 8814 mots grâce à l'objet Counter.<br>
</font>

In [ ]:
set(ck_productss['description'])

In [ ]:
mot = " ".join(set(ck_productss['description'])).split()

In [ ]:
len(" ".join(set(ck_productss['description'])).split())

In [ ]:
from collections import Counter
results = Counter(mot).most_common(8814)

In [ ]:
results

**Question 11:**

Consider all dataframes (you might concat them all into one after having processed them separately).

- which categories of products are sold most?
- which products are sold most?
- which products have the highest ratings?
- what are the top sellers (sold most and best ratings)?
- what is the top product color (sold most and best ratings)?

In [ ]:
df_all = pd.concat(list_data)
df_all

<font color="#0000ff">

Etapes :

1) Tout d'abord, on regarde dans chaque colonne du nouveau dataframe global le nombre de nan. On crée une variable new median qui applique la médiane des colonnes rating et review count d'un dictionnaire,
puis on remplit en particulier les colonnes du dataframe global avec la méthode fillna.<br>
2) Ensuite, on clean les brand ainsi que les catégory qui ne contiennent pas que des string. Pour cela, nous avons passé en revue toutes les colonnes et nous avons, 
dans un premier temps pour les brand, trié et regroupé chaque sous marques dans une marque plus 'large'; dans un second temps nous avons regroupé et trié tous 
les types de produits dans des catégories générales.

3) Enfin nous avons appliqué le même type de fonction afin de répondre aux mini questions du point 11):
Cette fonction prend en input un dataframe et selon les questions, un argument 'top number' afin de renseigner le nombre d'éléments en tête de liste que l'on veut faire apparaître.
On définit tout d'abord un nouveau mini dataframe composé des colonnes sélectionnées (qui dépendent selon les besoins des questions).<br>
On définit également une matrice vide. <br>
Ensuite chaque fonction parcourt le set des éléments de la colonne qui contient les données voulues (par exemple, product category si on étudit le type des produits),
puis va compléter la matrice vide des élements de cette colonne. On parcourt ensuite les éléments qui composent la matrice ordonnée alphabétiquement grâce à une boucle for et on 
sort les éléments voulus en les identifiant dans la matrice (item[1] par ex) avec un compteur à côté des éléments pour la mise en forme.
</font>

In [ ]:
new_median= df_all[['rating','review_count']].median()
df_all.fillna(value= new_median, inplace=True)

In [ ]:
df_all.isna().sum()

<font color="#ff0000">

Remarque :

Pour la fonction clean_brand décrite plus tôt, nous nous sommes rendues compte en parcourant que le dataframe Amazon contient certains produits qui ne correspondent pas à la marque indiquée. Par exemple, le produit de la marque btempted (sous-branche de Wacoal) référencé à l'index 29424 du dataframe est vendu sous la marque Nintendo. D'autres produits Wacoal comme le Wacoal Women's Balconette Bra est vendu sous la marque Fila. Par conséquent, nous avons décidé de regrouper toutes ces marques sous Wacoal puisqu'il s'agit dans tous les cas des produits Wacoal qui ont été mal référencés.
</font>

In [ ]:

def clean_brand(x):
    if 'ae' in x.lower():
        return 'American Eagle' 
    elif 'calvin' in x.lower():
        return 'Calvin Klein'
    elif'hanky' in x.lower():
        return 'Hanky Panky'
    elif 'victoria' in x.lower():
        return "Victoria's Secret"
    elif 'nordstrom' in x.lower():
        return 'NORDSTROM LINGERIE'
    elif 'topshop' in x.lower():
        return 'US TOPSHOP'
    elif 'vanity' in x.lower():
        return 'Vanity Fair'
    else:
        return'Wacoal Corporation'

In [ ]:
set(df_all['brand_name'].apply(clean_brand)) 

In [ ]:
def clean_category(x):
    if 'bikini' in x.lower() :
        return 'Bikinis'
    elif 'bridal' in x.lower():
        return 'Wedding only'
    elif 'bridget' in x.lower() or 'brooke' in x.lower() or 'audrey' in x.lower() or 'collections' in x.lower() or 'cheekies' in x.lower() or 'chill' in x.lower() or 'classic' in x.lower() or 'hannah' in x.lower() or 'katie' in x.lower():
        return 'Special collections'
    elif 'crop' in x.lower() or 'kimono' in x.lower() or 'onepiece' in x.lower() or 'one piece' in x.lower() or 'one-piece' in x.lower() or 'cross-back scoop'in x.lower() or 'tee' in x.lower() or 'tank' in x.lower() or 'top' in x.lower() or 'trim' in x.lower() or 'wrap' in x.lower() or 'neck' in x.lower():
        return 'Top'
    elif 'body' in x.lower() or 'bodies' in x.lower():
        return 'Bodies'
    elif 'bottle' in x.lower() or 'kit' in x.lower() or 'tote' in x.lower() or 'hat' in x.lower() or 'sock' in x.lower() or 'belt' in x.lower():
        return 'Accessories'
    elif 'sleep' in x.lower():
        return 'Sleepwear'
    elif 'sport' in x.lower() or 'gym' in x.lower():
        return 'Sport'
    else:
        return 'Lingerie'

<font color="#0000ff">

1) Pour les fonctions ci-dessus, nous avons défini des grands ensembles de catégories à la fois pour les catégories de produit et les marques. .<br>
2) Ci-dessous, on homogénéise les colonnes brand_name et product_category dans df_all car les différents dataframe n'ont pas forcément la même nomenclature.
</font>

In [ ]:
df_all['product_category']= df_all['product_category'].apply(clean_category)

In [ ]:
df_all['brand_name']= df_all['brand_name'].apply(clean_brand)

<font color="#0000ff">
Which categories of products are sold most?
</font>

<font color="#0000ff">

Pour cette question, il faut comprendre les produits "le plus vendu" comme ceux ayant le plus d'avis, c'est-à-dire qu'ils sont
très populaires et comportent le plus de "votes". Pour cela nous créons une fonction, qui comme expliqué précédemment, extrait les éléments 
des colonnes que l'on souhaite en parcourant le set des catégories de produits dans un nouveau dataframe (et en retirant au passage les doublons grâce à drop duplicates) puis les réinjecte dans une matrice vide selon cette disposition:
d'un côté, nos catégories de produits, de l'autre, les reviews qui correspondent à ces produits et qui sont comptés.
On réalise un peu de mise en forme pour l'affichage en demandant de sortir en output la succession de chaque catégorie avec le nombre de review à la ligne. 
Pour cela on crée une boucle for qui parcourt les catégories et les review dans la matrice ordonnée : on classe les catégories selon un ordre alphabétique.
</font>

In [ ]:
def best_category(dic,top_number): 
    
    dfn1 =dic.drop_duplicates(subset=['pdp_url'])[['product_category','review_count']] 
    matrice=[]
    
    for cat in set(dfn1['product_category']):
        matrice.append((cat, dfn1.loc[dfn1['product_category'].str.contains(cat,regex=False)]['review_count'].sum()))
        
    print(color.BOLD,"\nThe top",top_number,"most sold product categories is :\n",color.END)    
    
    for name,review in sorted(matrice, key=lambda item:(-item[1], item[0]))[:top_number]:
        print(sorted(matrice, key=lambda item: (-item[1], item[0])).index((name,review))+1,"-",name)

In [ ]:
best_category(df_all,5)

<font color="#0000ff">
Which products are sold most?
</font>

<font color="#0000ff">
Pour cette question, il suffit de faire la même chose que pour la question précédente sauf qu'on ne désire plus classer les catégories de produits selon leurs avis mais les produits eux-mêmes.
</font>

In [ ]:
def best_product(dic,top_number): 
    
    df2 =dic.drop_duplicates(subset=['pdp_url'])[['product_name','review_count']] 
    s=[]
    
    for product in set(df2['product_name']):
        s.append((product, df2.loc[df2['product_name'].str.contains(product,regex=False)]['review_count'].sum()))
        
    print(color.BOLD,"\nThe top",top_number,"most sold products is :\n",color.END)    
    
    for product,review in sorted(s, key=lambda item: (-item[1], item[0]))[:top_number]:
        print(sorted(s, key=lambda item: (-item[1], item[0])).index((product,review))+1,"-",product)

In [ ]:
best_product(df_all,5)

<font color="#0000ff">
Which products have the highest ratings?
</font>

<font color="#0000ff">
Pour cette question, on reprend le même fonctionnement que les précédentes questions sauf qu'ici, nous cherchons les produits les mieux notés.
Nous allons donc travailler sur les colonnes product_name et rating, mais nous n'allons cette fois-ci pas compter les notes attribuées 
à chaque produit car la somme fausserait les résultats. Nous faisons donc la moyenne des ratings sur chaque produit. Pour tous les produits identiques, on les classe sous forme alphabétique.
</font>

In [ ]:
def highest_rating(dic,top_number): 
    
    dfn3 =dic.drop_duplicates(subset=['pdp_url'])[['product_name','rating']] 
    matrix=[]
    
    for product in set(dfn3['product_name']):
        matrix.append((product, dfn3.loc[dfn3['product_name'].str.contains(product,regex=False)]['rating'].mean()))
        
    print(color.BOLD,"\nThe top",top_number,"highest rated products is :\n",color.END)    
    
    for product,rating in sorted(matrix, key=lambda item: (-item[1], item[0]))[:top_number]:
        print(sorted(matrix, key=lambda item: (-item[1], item[0])).index((product,rating))+1,"-",product)
        print("rating : ", rating)

In [ ]:
highest_rating(df_all,5)

<font color="#0000ff">
What are the top sellers (sold most and best ratings)?
</font>

<font color="#0000ff">
Pour cette question, on désire afficher les produits les plus vendus (cad avec le plus de reviews) et les mieux notés.
On fait la moyenne des rating sur chaque produit comme précédemment. Ensuite, on classe les produits selon les deux critères suivants:<br>
    > la note la plus élevée <br>
    > le nombre de reviews <br>
En effet, nous affichons le plus haut dans le classement, pour 2 produits ayant la meilleure note, celui qui a le plus de reviews. <br>
    A noter : pas besoin d'ordre alphabétique pour ordonner les produits qui sont déjà classés selon deux critères numériques
</font>

In [ ]:
def top_product(dic,top_number): 
    
    dfn4 =dic.drop_duplicates(subset=['pdp_url'])[['product_name','rating','review_count']] 
    matrixx=[]
    
    for product in set(dfn4['product_name']):
        matrixx.append((product, dfn4.loc[dfn4['product_name'].str.contains(product,regex=False)]['rating'].mean(), dfn4.loc[dfn4['product_name'].str.contains(product,regex=False)]['review_count'].sum()))
        
    print(color.BOLD,"\nThe top",top_number,"sellers are :\n",color.END)    
    
    for name,rating,review in sorted(matrixx, key=lambda item:(item[1],item[2]),reverse=True)[:top_number]:
        print(sorted(matrixx, key=lambda item: (item[1], item[2]),reverse=True).index((name,rating,review))+1,"-",name)
        print("rating :", rating)
        print("total reviews: ", review)    
        

In [ ]:
top_product(df_all,5)

<font color="#0000ff">
Pour cette question, on désire afficher cette fois-ci les couleurs les plus vendues (cad avec le plus de reviews) et les mieux notées.
On réalise le même principe qu'avant sauf qu'on travaille directement sur la colonne des couleurs.
</font>

In [ ]:
def top_product_color(dic,top_number): 
    
    dfn5 =dic.drop_duplicates(subset=['pdp_url'])[['color','rating','review_count']].dropna(subset=['color'])
    matris=[]
    
    for col in set(dfn5['color']):
        matris.append((col, dfn5.loc[dfn5['color'].str.contains(col,regex=False)]['rating'].mean(),dfn5.loc[dfn5['color'].str.contains(col,regex=False)]['review_count'].sum()))
        
    print(color.BOLD,"\nThe top",top_number,"colors are :\n",color.END)    
    
    for couleur,rating,review in sorted(matris, key=lambda item:(item[1], item[2]),reverse=True)[:top_number]:
        print(sorted(matris, key=lambda item: (item[1], item[2]),reverse=True).index((couleur,rating,review))+1,"-",couleur)
        print("rating: ", rating) 
        print("total reviews: ", review)  

In [ ]:
top_product_color(df_all,5)

**Question 12:**

Consider all dataframes (you might concat them all into one after having processed them separately).

- how can you assess an overall rating for a brand based on the ratings of its products?
- according to your definition, which brands have the best ratings?
- which types of products sold by all brands? (you might have to process the product category strings) and which product of these categories are sold by all brands?
- among these products, which brands has the best ratings?

<font color="#0000ff">
How can you assess an overall rating for a brand based on the ratings of its products? <br>
 According to your definition, which brands have the best ratings?
</font>

<font color="#0000ff">
Pour cette question, on réalise une fonction de même type que celles expliquées précédemment : <br>
    On désire évaluer et classer les notes de chaque marque en se fondant sur celles de leurs produits associés. On comprend tout de suite qu'on va devoir travailler sur les colonnes
    brand_name, product_name et rating. <br>
    Nous avons déjà défini une fonction clean_brand à la question 11 qui regroupe les sous-marques avec des noms alternatifs (comme Victoria's Secret Pink, en réalité sous marque de Victoria's Secret)
    pour faciliter le comptage. <br>
    La fonction définit un nouveau mini dataframe dfn6 composé des colonnes identifiées ainsi qu'une matrice vide, dans laquelle on ajoute les éléments contenus dans le set des marques de dfn6 ainsi que la moyenne de chaque marque.<br>
    On affiche ces dernières dans l'ordre alphabétique.
</font>

In [ ]:
def best_brand(dic,top_number): 
    
    dfn6 =dic.drop_duplicates(subset=['pdp_url'])[['product_name','brand_name','rating']] 
    matrixe=[]
    
    for brand in set(dfn6['brand_name']):
        matrixe.append((brand, dfn6.loc[dfn6['brand_name'].str.contains(brand,regex=False)]['rating'].mean()))
        
    print(color.BOLD,"\nThe top",top_number,"brands are :\n",color.END)    
    
    for name,rating in sorted(matrixe, key=lambda item:(-item[1], item[0]))[:top_number]:
        print(sorted(matrixe, key=lambda item: (-item[1], item[0])).index((name,rating))+1,"-",name)
        print("rating :", rating) 

In [ ]:
best_brand(df_all, 5)

In [ ]:
#déjà fait grâce à clean_category : on vérifit bien que les catégories fonctionnent
set(df_all['product_category'].apply(clean_category)) 

<font color="#0000ff">
Which types of products sold by all brands? (you might have to process the product category strings) and which product of these categories are sold by all brands?
</font>

<font color="#0000ff">
On désire trouver quels sont les types de produits communs à toutes les marques, par exemple si toutes les marques vendent des produits de Sport. 
Comme les types de produits ne sont pas forcément homogènes chez tous les retailers, on a définit une fonction clean_products qui, sur le même principe 
que les fonctions clean_brand et clean_category, essaie de lisser les types de produits pour qu'il n'y ait pas d'intersection vide (si jamais ce sont en réalité des produits de sport par exemple mais qu'ils n'ont pas la même appelation).<br>
Nous avons défini les catégories de produit suivantes : <br>
</font>

In [ ]:
def clean_products(x):
    if 'bikini' in x.lower() or 'bra' in x.lower() or 'bandeau' in x.lower() or 'triangle' in x.lower() or 'demi' in x.lower() or 'push' in x.lower() or 'petal' in x.lower() or 'bustier' in x.lower() or 'marquisette' in x.lower() or 'cami' in x.lower() or 'back' in x.lower() or 'cup' in x.lower() or 'teddy' in x.lower() or 'breath' in x.lower() or 'halter' in x.lower() or 'cheek' in x.lower() or 'strap' in x.lower() or 'penders' in x.lower() or 'band' in x.lower() or 'panty' in x.lower() or 'slip' in x.lower() or 'pant' in x.lower() or 'boy' in x.lower() or 'short' in x.lower() or 'leg' in x.lower() or 'undies' in x.lower() or 'brief' in x.lower() or 'thong' in x.lower() or 'hipster' in x.lower() or 'tanga' in x.lower() or 'knicker' in x.lower() or 'string' in x.lower() or 'kini' in x.lower() or 'tank' in x.lower() or 'garter' in x.lower():
        return 'Underwear'
    elif 'bodies' in x.lower() or 'body' in x.lower() or 'romper' in x.lower() :
        return 'Bodies'
    elif 'pajama' in x.lower() or 'night' in x.lower() or 'trim' in x.lower() or 'sleep' in x.lower() or 'dream' in x.lower():
        return 'Pyjamas'
    elif 'crop' in x.lower() or 'kimono' in x.lower() or 'onepiece' in x.lower() or 'one piece' in x.lower() or 'one-piece' in x.lower() or 'robe' in x.lower() or 'dress' in x.lower() or 'cross-back scoop'in x.lower() or 'tee' in x.lower() or 'tank' in x.lower() or 'top' in x.lower() or 'trim' in x.lower() or 'wrap' in x.lower() or 'chemise' in x.lower() or 'trunk' in x.lower() or 'hoodie' in x.lower() or 'sleev':
        return 'Top'
    elif 'accessories' in x.lower() or 'sock' in x.lower() or 'belt' in x.lower() or 'bottle' in x.lower() or 'neck'in x.lower() or 'mask' in x.lower() or 'kit' in x.lower() or 'beauty' in x.lower() or 'eye' in x.lower() or 'box' in x.lower() or 'basque' in x.lower():
        return 'Accessories'

<font color="#0000ff">
Ensuite, on applique clean_product à l'ensemble de la colonne product_name de df_all :
</font>

In [ ]:
df_all['product_name']= df_all['product_name'].apply(clean_products)

<font color="#0000ff">
La fonction ci-dessous common_product_cat fonctionne en 2 temps : <br>
    > en premier lieu, on cherche à afficher les catégories de produits vendues par toutes les marques, donc on va devoir
    travailler sur la colonne product_category. <br>
    > dans un second temps, on désire afficher quels sont les produits dans les catégories affichées qui sont également vendus par toutes les marques : c'est comme
        si on réalisait un affinement de la première étape<br>
    
   On définit la fonction common_product_and_cat qui va chercher les catégories communes à toutes les marques. Pour se faire on parcourt les product_category et on regarde si notre catégorie est vendue par toutes les marques (en regardant le nombre de marque et la longueur de notre liste). Ensuite, toujours selon le même raisonnement mais cette fois-ci parmi les produits de cette catégorie, on cherche les produits qui sont vendus par toutes les marques.


A noter : Nous avons effectué cette classification dans les colonnes product_category et product_category afin d'éviter d'obtenir une intersection vide dans le résultat demandé à la question 12. Mais il faut garder à l'esprit qu'il n'existe pas une réponse parfaite et qu'ici, une analyse qui retourne les catégories présentes et les produits de cette catégorie présentés pour toutes les marques a été privilégié. Parce que la classification des catégories effectuée avec clean_category et la classification des produits avec clean_product, le résultat peut varier et on peut très bien n'obtenir aucun produit vendu par toutes les marques. <br> Il faut alors rester prudent sur l'interprétation de cette donnée étant donné quelle découle d'un choix arbitraire. De plus, ces choix sont successifs ce qui rend encore plus possible le fait d'obtenir un résultat faussé. 
</font>

In [ ]:
def common_product_and_cat(df):
    df_cat_brands = df[['brand_name','product_category']].drop_duplicates()
    for cat in set(df['product_category']):
        if list(df_cat_brands['product_category']).count(cat) == len(set(df['brand_name'])):
            print(cat, "category is sold by all brands.\n")
            df_p = df.loc[df['product_category'].str.contains(cat, regex = False)][['brand_name', 'product_name']].drop_duplicates()
            p = []
            for product in set(df_p['product_name']):
                if list(df_p['product_name']).count(product) == len(set(df_p['brand_name'])):
                    p.append(product)
                    print(product,"in", cat,"category is sold by all brands.\n", color.END)
            if len(p) == 0:
                print("There is no product in", cat, "category which is sold by all brands.")

In [ ]:
common_product_and_cat(df_all)

Among these products, which brands has the best ratings?

<font color="#0000ff">
Sur le même principe que pour highest_rating (cf. question 11), cette fonction travaille sur la catgéorie de produits Underwear car on a trouvé précédemment qu'il s'agissait de la seule vendue par toutes les marques.
</font>

In [ ]:
def brand_rating_underwear(dic,top_brands):
    df_bis = dic.loc[dic['product_name'].str.contains('Underwear', regex = False)][['product_name','brand_name','rating']]
    means = []
    count = 0

    for brand in set(dic['brand_name']):
        means.append((brand, df_bis.loc[df_bis['brand_name'].str.contains(brand,regex=False)]['rating'].mean()))
    
    print("The top",top_brands,"brands for Underwear is :\n", color.END) 

    for name,rating in sorted(means, key=lambda item: item[1],reverse=True)[:top_brands]:
        count += 1
        print(count,'-', name, ":")
        print('Rating :',rating)


In [ ]:
brand_rating_underwear(df_all,5)